In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import datetime as dt

import pickle
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

path = '/content/drive/MyDrive/3rd Sem/Thesis-Cloud/Latency_Word/Dataset_Agnostic/' + 'df_latency_word(SBU_Proposed).pkl'

df_final  = pd.read_pickle(path)
df=df_final.copy()

# Extract features and target
X = df.iloc[:, :-1]  # Features (all columns except the last one)
y = df.iloc[:, -1]   # Target (last column)

X_train = X
y_train = y
print("Train data shape: ", X_train.shape, y_train.shape)

# X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=.3,random_state =123, stratify=y)

Train data shape:  (22640, 210) (22640,)


In [ ]:
path = '/content/drive/MyDrive/3rd Sem/Thesis-Cloud/Latency_Word/Dataset_Agnostic/' + 'df_latency_word(Buffalo).pkl'
df_test = pd.read_pickle(path)

X_test = df_test.iloc[:, :-1]
y_test = df_test.iloc[:, -1]
print("Test data shape: ", X_test.shape, y_test.shape)

Test data shape:  (14847, 210) (14847,)


# LGBMClassifier

In [ ]:
ts_now = dt.datetime.now()
print(ts_now)

2024-05-14 21:03:05.836430


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV

import lightgbm as lgb
from lightgbm import LGBMClassifier
import pickle

In [ ]:
model_path = path = '/content/drive/MyDrive/3rd Sem/Thesis-Cloud/Latency_Word/Dataset_Agnostic/'

In [ ]:
# Define parameter grid
param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.5],
    'max_depth': [3, 5, 7, 10],
    'num_leaves': [31, 50, 100],
    'boosting_type': ['gbdt', 'dart', 'goss']
}

# Create LGBMClassifier instance
lgbm = LGBMClassifier()

# Initialize GridSearchCV with n_jobs=-1 to use all available cores
grid_search_lgbm = GridSearchCV(lgbm, param_grid, refit=True, verbose=2, cv=5, scoring='accuracy', n_jobs=-1)

# Fit GridSearchCV
grid_search_lgbm.fit(X_train, y_train)

# Best parameter after tuning
print("Best Parameters: ", grid_search_lgbm.best_params_)

# Model Performance
print("Best Estimator Accuracy: ", grid_search_lgbm.best_score_)

best_model_filename = model_path+'SBU_Proposed_Dataset_LGBM_Classifier.pkl'

with open(best_model_filename, 'wb') as file:
    pickle.dump(grid_search_lgbm.best_estimator_, file)

print(f"Best model saved to {best_model_filename}")
print()

# To make predictions with the saved model, load it from the disk:

# Make predictions on the test set
y_pred = grid_search_lgbm.predict(X_test)

# Evaluate the accuracy
print()
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

# Calculate F1 score
from sklearn.metrics import f1_score
f1 = f1_score(y_test, y_pred)
print("F1 Score:", f1)

def calculate_FAR_FRR(y_true, y_pred):
    # Calculate confusion matrix
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()

    # Calculate FAR and FRR
    FRR = fp / (fp + tn)
    FAR = fn / (fn + tp)

    print(fp, "FRR: ", FRR)
    print(fn, "FAR: ", FAR)

    return FAR, FRR

calculate_FAR_FRR(y_test, y_pred)

Fitting 5 folds for each of 324 candidates, totalling 1620 fits
[LightGBM] [Info] Number of positive: 9748, number of negative: 12892
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.053516 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 39338
[LightGBM] [Info] Number of data points in the train set: 22640, number of used features: 180
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.430565 -> initscore=-0.279545
[LightGBM] [Info] Start training from score -0.279545
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Best Parameters:  {'boosting_type': 'gbdt', 'learning_rate': 0.1, 'max_depth': 10, 'n_estimators': 200, 'num_leaves': 50}
Best Estimator Accuracy:  0.8523851590106007
Best model saved to /content/drive/MyDrive/3rd Sem/Thesis-Cloud/Latency_Word/Dataset_Agnostic/SBU_Proposed_Dataset_LGBM_Classifier.pkl


Accuracy: 0.719876069239577
F1 Score: 0.7574219889180519
591 F

(0.35463671603220354, 0.12348516506477225)

In [ ]:
ts_now = dt.datetime.now()
print(ts_now)

2024-05-14 22:55:44.439719
